In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/elomia

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/elomia'
/content/drive/MyDrive/elomia


# TPU configuration

In [ ]:
# import tensorflow as tf
# import os

# assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

# assert float('.'.join(tf.__version__.split('.')[:2])) >= 1.14, 'Make sure that Tensorflow version is at least 1.14'

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))

# strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 17.8 MB/s 
     |████████████████████████████████| 6.6 MB 32.0 MB/s 
     |████████████████████████████████| 596 kB 14.6 MB/s 
     |████████████████████████████████| 77 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 46.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 11.0 MB/s 
     |████████████████████████████████| 136 kB 59.7 MB/s 
     |████████████████████████████████| 1.1 MB 22.2 MB/s 
     |████████████████████████████████| 212 kB 16.3 MB/s 
     |████████████████████████████████| 127 kB 46.9 MB/s 
     |████████████████████████████████| 144 kB 51.9 MB/s 
     |████████████████████████████████| 271 kB 60.3 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

emd_df = pd.read_csv("API/ParlAI_ex/emp_data_train.csv", on_bad_lines='skip')
emd_df['utterance'] = emd_df['utterance'].str.lower()
emd_df['prompt'] = emd_df['prompt'].str.lower()

emd_df.reset_index(inplace=True)

In [ ]:
import numpy as np

slice_most_repeated_indexes = emd_df['speaker_idx'].value_counts()[0:20].index

most_repeated = emd_df[emd_df['speaker_idx'].isin(slice_most_repeated_indexes)]

most_repeated["two_input"] = most_repeated[['utterance', 'prompt']].apply(lambda x: ' '.join(x), axis=1)

unq_speakers = most_repeated["speaker_idx"].unique().tolist()
print(len(unq_speakers))
most_repeated["speaker_label"] = list(map(lambda x: unq_speakers.index(x), most_repeated["speaker_idx"]))

unq_contexts = most_repeated["context"].unique().tolist()
print(len(unq_contexts))

most_repeated["context_label"] = list(map(lambda x: unq_contexts.index(x), most_repeated["context"]))

20
32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [ ]:
train_data = most_repeated[["two_input", "speaker_label", "context_label"]]
print(len(train_data["two_input"]))
print(len(train_data[["speaker_label", "context_label"]]))

27482
27482


In [ ]:
len(train_data["two_input"].sort_values()[0])
# print(train_data["two_input"].iloc[60438]))

327

In [ ]:
from sklearn.model_selection import train_test_split


X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(train_data["two_input"],
                                                          train_data[["speaker_label", "context_label"]], 
                                                          test_size=0.3)


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, TFBertModel

model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

#with strategy.scope():

bert = TFBertModel.from_pretrained(model_name)

print(len(bert.bert.encoder.layer))

for i in range(len(bert.bert.encoder.layer)):
    bert.bert.encoder.layer[i].trainable = i >= 10

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


12


In [ ]:
max_length = 327 

x_train = tokenizer(
    text = X_train_s.tolist(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

x_val= tokenizer(
    text= X_val_s.tolist(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

import tensorflow as tf


In [ ]:
# with strategy.scope():
max_len = max_length
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

embeddings = bert(input_ids,attention_mask = input_mask)[0] 

out = tf.keras.layers.GlobalMaxPool1D()(embeddings)

out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation = 'relu')(out)

y1 = Dense(32, activation = 'softmax', name="context")(out) 
y2 = Dense(20, activation = 'softmax', name="speaker")(out) 


model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=[y1, y2])
model.layers[2].trainable = True

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_cont = to_categorical(y_train_s["context_label"])
y_train_speaker = to_categorical(y_train_s["speaker_label"])

y_val_cont = to_categorical(y_val_s["context_label"])
y_val_speaker = to_categorical(y_val_s["speaker_label"])

In [32]:
from keras.models import load_model
import os 

# with strategy.scope():
optimizer = Adam(learning_rate=5e-05, #e--5
                  decay=0.01,
                  clipnorm=1.0)

loss = CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy')

model.compile(optimizer = optimizer, loss = loss, metrics = metric)
model.load_weights("/content/drive/MyDrive/elomia/API/ParlAI_ex/bert_logs/two_inp/checkpoint/checkpoints")
# model.run_eagerly = True

NotFoundError: ignored

In [ ]:
import os
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

tf.compat.v1.enable_eager_execution()

exp_path = './API/ParlAI_ex/bert_logs/two_inp/checkpoint'

tensoboard_callback = TensorBoard(log_dir=exp_path)
checkpoint_callback = ModelCheckpoint(os.path.join(exp_path,"checkpoints"),
                                      monitor='val_loss', 
                                      verbose=1,
                                      mode='min',
                                      save_weights_only=False,
                                      save_best_only=True)



# with strategy.scope():

train_history = model.fit(
    
    x = {'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = [y_train_cont, y_train_speaker],

    validation_data = (
    {'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']},
      [y_val_cont, y_val_speaker]
    ),

    callbacks=[tensoboard_callback, checkpoint_callback],
    epochs=50,
    batch_size=20,
    shuffle=True
)



    # epochs = 2
    # for epoch in range(epochs):
    #     print("\nStart of epoch %d" % (epoch,))

    #     for step, (x_batch_train, y_batch_train) in enumerate(zip({'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']}, [y_train_cont, y_train_speaker])):

    #         # Open a GradientTape to record the operations run
    #         # during the forward pass, which enables auto-differentiation.
    #         with tf.GradientTape() as tape:

    #             # Run the forward pass of the layer.
    #             # The operations that the layer applies
    #             # to its inputs are going to be recorded
    #             # on the GradientTape.
    #             print(x_batch_train)
    #             logits = model(x_batch_train, training=True)  # Logits for this minibatch

    #             # Compute the loss value for this minibatch.
    #             loss_value = loss_fn(y_batch_train, logits)

    #         # Use the gradient tape to automatically retrieve
    #         # the gradients of the trainable variables with respect to the loss.
    #         grads = tape.gradient(loss_value, model.trainable_weights)

    #         # Run one step of gradient descent by updating
    #         # the value of the variables to minimize the loss.
    #         optimizer.apply_gradients(zip(grads, model.trainable_weights))

    #         # Log every 200 batches.
    #         if step % 200 == 0:
    #             print(
    #                 "Training loss (for one batch) at step %d: %.4f"
    #                 % (step, float(loss_value))
    #             )
    #             print("Seen so far: %s samples" % ((step + 1) * batch_size))


Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


962/962 [==============================] - ETA: 0s - loss: 6.0198 - context_loss: 3.1936 - speaker_loss: 2.8262 - context_balanced_accuracy: 0.1465 - speaker_balanced_accuracy: 0.1485
Epoch 1: val_loss improved from inf to 5.58525, saving model to ./API/ParlAI_ex/bert_logs/two_inp/checkpoint/checkpoints


INFO:tensorflow:Assets written to: ./API/ParlAI_ex/bert_logs/two_inp/checkpoint/checkpoints/assets


INFO:tensorflow:Assets written to: ./API/ParlAI_ex/bert_logs/two_inp/checkpoint/checkpoints/assets


962/962 [==============================] - 1743s 2s/step - loss: 6.0198 - context_loss: 3.1936 - speaker_loss: 2.8262 - context_balanced_accuracy: 0.1465 - speaker_balanced_accuracy: 0.1485 - val_loss: 5.5853 - val_context_loss: 2.9053 - val_speaker_loss: 2.6800 - val_context_balanced_accuracy: 0.2429 - val_speaker_balanced_accuracy: 0.2239
Epoch 2/50
962/962 [==============================] - ETA: 0s - loss: 5.4357 - context_loss: 2.8119 - speaker_loss: 2.6238 - context_balanced_accuracy: 0.2675 - speaker_balanced_accuracy: 0.2402
Epoch 2: val_loss improved from 5.58525 to 5.32165, saving model to ./API/ParlAI_ex/bert_logs/two_inp/checkpoint/checkpoints


INFO:tensorflow:Assets written to: ./API/ParlAI_ex/bert_logs/two_inp/checkpoint/checkpoints/assets


INFO:tensorflow:Assets written to: ./API/ParlAI_ex/bert_logs/two_inp/checkpoint/checkpoints/assets


962/962 [==============================] - 1727s 2s/step - loss: 5.4357 - context_loss: 2.8119 - speaker_loss: 2.6238 - context_balanced_accuracy: 0.2675 - speaker_balanced_accuracy: 0.2402 - val_loss: 5.3216 - val_context_loss: 2.7077 - val_speaker_loss: 2.6139 - val_context_balanced_accuracy: 0.2931 - val_speaker_balanced_accuracy: 0.2484
Epoch 3/50
693/962 [====================>.........] - ETA: 6:37 - loss: 5.2043 - context_loss: 2.6621 - speaker_loss: 2.5422 - context_balanced_accuracy: 0.3022 - speaker_balanced_accuracy: 0.2613

In [ ]:
# https://www.analyticsvidhya.com/blog/2021/12/multiclass-classification-using-transformers/

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import get_scheduler
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3

num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(name="linear",
                             optimizer=optimizer,
                             num_warmup_steps=0,
                             num_training_steps=num_training_steps)


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(1):

    for ind, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)

        loss = outputs.loss
        if ind%300==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        loss.backward()

        optimizer.step()

        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)

  0%|          | 0/10308 [00:00<?, ?it/s]

Epoch: 0, Loss:  3.355130434036255


KeyboardInterrupt: ignored

In [ ]:


def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(train_dataloader, 0):
            data = {k: v.to(device) for k, v in data.items()}
            outputs = model(**data)

            targets = data['labels'].to(device, dtype = torch.float)

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs.logits).cpu().detach().numpy().tolist())
            break
    return fin_outputs, fin_targets



In [ ]:


for epoch in range(1):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")



NameError: ignored